# AI Bootcamp 04 - Multi-Domain Agent System

In [1]:
import os
from openai import OpenAI

In [2]:
# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", "YOUR_API_KEY"))

In [3]:
# ------------------ SHARED MEMORY ------------------
class SharedMemory:
    """Stores shared task context across multiple agents."""
    def __init__(self):
        self.memory = {}

    def store(self, key, value):
        self.memory[key] = value

    def retrieve(self, key):
        return self.memory.get(key, None)

shared_memory = SharedMemory()

# ------------------ DOMAIN AGENTS ------------------
class BaseAgent:
    """Base class for all agents."""
    def __init__(self, name):
        self.name = name

    def process_request(self, request, context):
        raise NotImplementedError("Each agent must implement this method.")

class ResearchAgent(BaseAgent):
    """Retrieves relevant knowledge and best practices."""
    def __init__(self):
        super().__init__("ResearchAgent")

    def process_request(self, request, context):
        prompt = f"Retrieve relevant research and best practices for: {request}"
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=300
        )
        result = response.choices[0].message.content.strip()
        shared_memory.store("research", result)
        return result

class CodingAgent(BaseAgent):
    """Writes code based on research findings."""
    def __init__(self):
        super().__init__("CodingAgent")

    def process_request(self, request, context):
        research_data = context.get("research", "No research data available.")
        prompt = (
            f"Write Python code for the following task: {request}\n"
            f"Follow the best practices from this research:\n{research_data}"
        )
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=500
        )
        result = response.choices[0].message.content.strip()
        shared_memory.store("code", result)
        return result

class TestingAgent(BaseAgent):
    """Generates test cases for the produced code."""
    def __init__(self):
        super().__init__("TestingAgent")

    def process_request(self, request, context):
        code_snippet = context.get("code", "No code available.")
        prompt = f"Generate unit tests in Python for the following code:\n{code_snippet}"
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=300
        )
        result = response.choices[0].message.content.strip()
        shared_memory.store("tests", result)
        return result

# ------------------ AGENT RUNNER ------------------
class AgentRunner:
    """Manages agents and assigns tasks dynamically."""
    def __init__(self):
        self.agents = {
            "research": ResearchAgent(),
            "coding": CodingAgent(),
            "testing": TestingAgent(),
        }
        self.current_agent = None

    def reassign_agent(self, task_stage):
        """Determines which agent should handle the request."""
        if task_stage == "research":
            self.current_agent = self.agents["research"]
        elif task_stage == "coding":
            self.current_agent = self.agents["coding"]
        elif task_stage == "testing":
            self.current_agent = self.agents["testing"]

    def execute_task(self, request, stage):
        """Runs the assigned agent and transfers the task if needed."""
        self.reassign_agent(stage)
        print(f"\n🔹 Active Agent: {self.current_agent.name}")

        context = shared_memory.memory  # Retrieve shared task context
        response = self.current_agent.process_request(request, context)
        print(f"\n✅ {self.current_agent.name} Output:\n{response}")
        return response



In [4]:
# ------------------ SYSTEM EXECUTION ------------------
if __name__ == "__main__":
    agent_runner = AgentRunner()

    # Example User Request
    user_request = "Write a function in Python to calculate the factorial of a number."

    # Step 1: Research Agent retrieves best practices
    research_output = agent_runner.execute_task(user_request, "research")

    # Step 2: Coding Agent writes the function based on research
    code_output = agent_runner.execute_task(user_request, "coding")

    # Step 3: Testing Agent generates test cases for the function
    test_output = agent_runner.execute_task(user_request, "testing")

    print("\n🔹 Final Code:\n", code_output)
    print("\n🔹 Generated Test Cases:\n", test_output)



🔹 Active Agent: ResearchAgent

✅ ResearchAgent Output:
Here is a simple Python function to calculate the factorial of a number:

```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```

To use this function, simply call it with the desired number as an argument. For example:

```python
result = factorial(5)
print(result)
```

This will output `120`, which is the factorial of 5.

It's important to note that this function uses recursion to calculate the factorial. While recursion is a common and elegant way to solve this problem, it may not be the most efficient for very large numbers. In such cases, it may be better to use an iterative approach.

Here are some best practices to keep in mind when writing a function to calculate the factorial of a number in Python:

1. Handle edge cases: Make sure to handle edge cases such as when the input number is 0 or negative.

2. Use recursion or iteration: Choose the appropriate method (recursion